In [5]:
import pandas as pd
import numpy as np
import glob

In [9]:
data_folder = "data"
csvDataFiles = glob.glob(data_folder + "/*.csv")
dataFiles = []
for dataFile in csvDataFiles:
    df = pd.read_csv(dataFile)
    dataFiles.append(df)

In [10]:
#The number of stocks we have in our data folder
print(len(dataFiles))

30


In [11]:
#This is what our data looks like
dataFiles[0].head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-10-02,52.160000,52.209999,51.660000,51.869999,51.542397,7516200
1,2017-10-03,52.000000,52.049999,51.400002,51.470001,51.144924,8126700
2,2017-10-04,51.509998,52.130001,51.400002,52.080002,51.751072,8204300
3,2017-10-05,52.009998,52.320000,51.779999,52.180000,51.850441,5287400
4,2017-10-06,52.200001,52.459999,52.119999,52.419998,52.088921,5360300


In [12]:
#Lets store the ratings in a form we can understand
ratings_folder = "Ratings"
xlsxRatingFiles = glob.glob(ratings_folder + "/*.xlsx")
ratingsFiles = []
for ratingFile in xlsxRatingFiles:
    rf = pd.ExcelFile(ratingFile)
    ratingsFiles.append(rf)

In [13]:
#The number of ratings files we have in our ratings folder
print(len(ratingsFiles))

32


In [17]:
#This is what our ratings look like
ratingsFiles[0].head()

AttributeError: 'ExcelFile' object has no attribute 'head'